In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Causal_Inference_Model/

/content/drive/MyDrive/Causal_Inference_Model


In [5]:
import sys, os

In [6]:
sys.path.append(os.path.abspath('scripts/'))

In [8]:
!pip install causalnex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 5.2 MB/s 
     |████████████████████████████████| 27.4 MB 78.6 MB/s 
     |████████████████████████████████| 1.9 MB 56.0 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 22.3 MB 41.8 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 115 kB 64.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=055447acc1628368407a43a977a023d8e1932af155f3151f99a7fc52b1c842f8
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: scikit-learn
    Found existing in

Import Libraries

In [11]:
import warnings
import pandas as pd
from causalnex.structure import StructureModel
from causal_pipeline import EDAPipeline
from causal_helper import CausalHelper
warnings.filterwarnings("ignore")

In [13]:
preprocessed_trip_df = pd.read_csv("data/processed_trip.csv")
preprocessed_trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534369 entries, 0 to 534368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        534369 non-null  int64  
 1   Trip ID           534369 non-null  int64  
 2   Trip Origin       534369 non-null  object 
 3   Trip Destination  534369 non-null  object 
 4   Trip Start Time   534369 non-null  object 
 5   Trip End Time     534368 non-null  object 
 6   is_weekend        534369 non-null  int64  
 7   is_holiday        534369 non-null  int64  
 8   distance          534369 non-null  float64
 9   speed             534369 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 40.8+ MB


In [14]:
#Load the second data
df_driver = pd.read_csv("data/driver_locations_during_request.csv")
df_driver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


#### Merge the two Datasets

In [15]:
preprocessed_trip_df['driver_distance'] = preprocessed_trip_df['distance'].apply(lambda x: -1)
preprocessed_trip_df['driver_lat'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)
preprocessed_trip_df['driver_lng'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)